In [1]:
import os
from pathlib import Path

import pandas as pd

from util import with_spotify_track_metadata

diff_dataset = True
data_path = Path("../../data").resolve()
os.makedirs(data_path, exist_ok=True)

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def fetch_song_chart_by_month():
    """Chart 2000 - 2021: the top 50 chart for every month from Jan 2000 to Jun 2021
    Note: song and albums ranked weekly by indicative revenue globally over a certain period. Indicative Revenue is a model for global sales volume, adjusted for inflation and currency exchange, and using IFPI estimated annual revenue for the recorded music industry in each country.
    #songs_chart_by_month = pd.read_csv("https://chart2000.com/data/chart2000-songmonth-0-3-0063.csv")
    """
    return pd.read_csv("/Users/pez/Downloads/chart2000-songmonth-0-3-0063.csv")


songs_chart_by_month = fetch_song_chart_by_month()
song_artist_df = songs_chart_by_month[["song", "artist"]].drop_duplicates()
song_artist_df = with_spotify_track_metadata(song_artist_df)
#songs_chart_by_month.to_parquet(data_path / "raw/song_chart_by_month.pq", index=False)
#songs_chart_by_month.to_csv(data_path / "song_chart_by_month.csv.gz", compression="gzip", index=False)

status: 0.23 (8 / 3454
status: 0.90 (31 / 3454
status: 1.13 (39 / 3454
status: 1.88 (65 / 3454
status: 2.20 (76 / 3454
status: 2.29 (79 / 3454
status: 2.69 (93 / 3454
status: 2.81 (97 / 3454
status: 3.13 (108 / 3454
status: 3.91 (135 / 3454
status: 4.20 (145 / 3454
status: 4.46 (154 / 3454
status: 4.86 (168 / 3454
status: 5.30 (183 / 3454
status: 5.44 (188 / 3454
status: 5.79 (200 / 3454
status: 5.82 (201 / 3454
status: 6.14 (212 / 3454
status: 6.46 (223 / 3454
status: 6.72 (232 / 3454
status: 6.89 (238 / 3454
status: 7.12 (246 / 3454
status: 7.30 (252 / 3454
status: 7.50 (259 / 3454
status: 8.54 (295 / 3454
status: 8.71 (301 / 3454
status: 9.03 (312 / 3454
status: 9.09 (314 / 3454
status: 9.29 (321 / 3454
status: 9.58 (331 / 3454
status: 9.99 (345 / 3454
status: 10.63 (367 / 3454
status: 10.83 (374 / 3454
status: 11.12 (384 / 3454
status: 11.61 (401 / 3454
status: 11.96 (413 / 3454
status: 12.28 (424 / 3454
status: 12.48 (431 / 3454
status: 12.57 (434 / 3454
status: 12.88 (445 / 3454


In [6]:
song_artist_df[song_artist_df["spotify_track_id"].isnull()]

,song,artist,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
0,Smooth,Rob Thomas & Santana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Blue (Da Ba Dee),Eiffel 65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Hot Boyz,"Missy 'Misdemeanor' Elliott, NAS & Eve",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Bring It All To Me,Blaque,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Satisfy You,P Diddy & R Kelly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12846,Bed,Joel Corry & RAYE & David Guetta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12872,my.life,"J Cole, 21 Savage & Morray",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12881,Every Chance I Get,DJ Khaled & Lil Baby & Lil Durk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12884,pride.is.the.devil,J. Cole & Lil Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
songs_chart_by_month

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au
0,Jan 2000,1,Rob Thomas & Santana,Smooth,3911.953,1,-,44,-,-,5
1,Jan 2000,2,Christina Aguilera,What A Girl Wants,3787.015,1,81,18,-,1,9
2,Jan 2000,3,Savage Garden,I Knew I Loved You,3693.844,1,56,68,43,1,21
3,Jan 2000,4,Celine Dion,That's The Way It Is,3341.488,8,20,9,10,-,14
4,Jan 2000,5,Eiffel 65,Blue (Da Ba Dee),3340.978,6,28,22,10,1,1
...,...,...,...,...,...,...,...,...,...,...,...
12895,Jun 2021,46,Ariana Grande,POV,849.983,33,-,-,-,-,-
12896,Jun 2021,47,Dierks Bentley,Gone,843.855,32,-,-,-,85,-
12897,Jun 2021,48,Olivia Rodrigo,"Jealousy, Jealousy",838.863,24,-,-,-,21,22
12898,Jun 2021,49,Olivia Rodrigo,"1 Step Forward, 3 Steps Back",837.277,19,-,-,-,17,18


In [3]:
songs_chart_by_month = fetch_song_chart_by_month()
bsb_songs_chart = songs_chart_by_month[songs_chart_by_month["artist"].str.contains("Backstreet Boys")]
bsb_songs_chart = with_spotify_track_metadata(bsb_songs_chart)
bsb_songs_chart

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
43,Jan 2000,44,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1457.522,16,97,-,-,-,-,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
56,Feb 2000,7,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2914.772,10,3,3,-,1,20,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
102,Mar 2000,3,The Backstreet Boys,Show Me The Meaning Of Being Lonely,3678.622,6,3,4,-,1,19,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
158,Apr 2000,9,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2945.213,6,28,22,-,-,23,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
219,May 2000,20,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1919.076,19,64,66,-,-,41,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
275,Jun 2000,26,The Backstreet Boys,The One,1857.997,31,8,15,-,-,-,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
321,Jul 2000,22,The Backstreet Boys,The One,1997.071,30,15,41,-,-,41,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
504,Nov 2000,5,The Backstreet Boys,Shape Of My Heart,3351.040,10,4,2,-,1,5,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
552,Dec 2000,3,The Backstreet Boys,Shape Of My Heart,3408.347,9,20,3,-,1,11,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
619,Jan 2001,20,The Backstreet Boys,Shape Of My Heart,2208.448,19,49,14,-,-,17,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day


In [5]:
import json
from time import sleep

from util import spotify


def fetch_bulk_track_features(track_ids):
    fetched_tracks = []
    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        if i % 500 == 0:
            print(f"fetching track metadata: {i} ({i / len(track_ids) * 100:.2f})")
        tracks = spotify.tracks(chunk)["tracks"]
        sleep(.50)
        track_audio_features = spotify.audio_features(chunk)
        sleep(.50)
        for track, audio_features in zip(tracks, track_audio_features):
            track_ = {
                "id": track["id"],
                "name": track["name"],
                "album": track["album"]["name"],
                "album_id": track["album"]["id"],
                "artists": json.dumps([a["name"] for a in track["artists"]]),
                "artist_ids": json.dumps([a["id"] for a in track["artists"]]),
                "track_number": track["track_number"],
                "disc_number": track["disc_number"],
                "explicit": track["explicit"],
                "duration_ms": track["duration_ms"],
                "year": int(track["album"]["release_date"][0:4]),
                "release_date": track["album"]["release_date"],
                "popularity": track["popularity"],
                "audio_features": 0,
                "isrc": track.get("external_ids", {}).get("isrc")
            }
            if audio_features is not None:
                track_.update({
                    "audio_features": 1,
                    "danceability": audio_features["danceability"],
                    "energy": audio_features["energy"],
                    "key": audio_features["key"],
                    "loudness": audio_features["loudness"],
                    "mode": audio_features["mode"],
                    "speechiness": audio_features["speechiness"],
                    "acousticness": audio_features["acousticness"],
                    "instrumentalness": audio_features["instrumentalness"],
                    "liveness": audio_features["liveness"],
                    "valence": audio_features["valence"],
                    "tempo": audio_features["tempo"],
                    "time_signature": audio_features["time_signature"],
                })
            fetched_tracks.append(track_)
    return pd.DataFrame(fetched_tracks)

track_ids = bsb_songs_chart["spotify_track_id"].tolist()
tracks_df = fetch_bulk_track_features(track_ids)
tracks_df

fetching track metadata: 0 (0.00)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
1,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
2,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
3,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
4,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
5,44kgtDJyAFv4Z01z7bIxco,The One,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",8,1,False,226933,...,5,-4.227,0,0.0348,0.0646,0,0.1640,0.621,110.031,4
6,44kgtDJyAFv4Z01z7bIxco,The One,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",8,1,False,226933,...,5,-4.227,0,0.0348,0.0646,0,0.1640,0.621,110.031,4
7,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4
8,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4
9,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4


In [6]:
bsb_songs_chart

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
43,Jan 2000,44,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1457.522,16,97,-,-,-,-,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
56,Feb 2000,7,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2914.772,10,3,3,-,1,20,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
102,Mar 2000,3,The Backstreet Boys,Show Me The Meaning Of Being Lonely,3678.622,6,3,4,-,1,19,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
158,Apr 2000,9,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2945.213,6,28,22,-,-,23,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
219,May 2000,20,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1919.076,19,64,66,-,-,41,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
275,Jun 2000,26,The Backstreet Boys,The One,1857.997,31,8,15,-,-,-,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
321,Jul 2000,22,The Backstreet Boys,The One,1997.071,30,15,41,-,-,41,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
504,Nov 2000,5,The Backstreet Boys,Shape Of My Heart,3351.040,10,4,2,-,1,5,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
552,Dec 2000,3,The Backstreet Boys,Shape Of My Heart,3408.347,9,20,3,-,1,11,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
619,Jan 2001,20,The Backstreet Boys,Shape Of My Heart,2208.448,19,49,14,-,-,17,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day


In [42]:
song_artist_df_2 = song_artist_df[song_artist_df["spotify_track_id"].isnull()]

In [44]:
from time import sleep

from thefuzz import fuzz
from requests.exceptions import ReadTimeout

from util import spotify, resolve_names, filter_by_track


def best_match(result, artist, track):
    for t in result["tracks"]["items"]:
        t_artist = t["artists"][0]["name"]
        t_track = t["name"]
        if "karaoke" in t_artist.lower() or "karaoke" in t_track.lower():
            continue
        if "version" in t_track.lower() and not "version" in track.lower():
            continue
        artist_fuzz_score = fuzz.token_set_ratio(artist, t_artist)
        track_fuzz_score = fuzz.token_set_ratio(track, t_track)
        compound_fuzz_score = fuzz.token_set_ratio(f"{artist} {track}", f"{t_artist} {t_track}")
        # print(artist, t_artist, artist_fuzz_score, track, t_track, track_fuzz_score, compound_fuzz_score)
        if (artist_fuzz_score > 85 and track_fuzz_score > 85) or compound_fuzz_score > 90:
            return t
    return None


def fetch_track(artist, track):
    artists = [artist]
    if artist.startswith("The "):
        artists.append(artist[4:])
    for artist in artists:
        try:
            artist_ = artist
            result = spotify.search(f"track:{track} artist:{artist}")
            sleep(.25)
            best_match_ = best_match(result, artist, track)
            if result["tracks"]["total"] == 0 or not best_match_:
                artist_, _ = resolve_names(artist, track)
                result = spotify.search(f"track:{track} artist:{artist_}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist_} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                return None
        except ReadTimeout:
            print("spotify time out.")
            return None
        if best_match_:
            return best_match_
    return None


def with_spotify_track_metadata(df, artist_key="artist", track_key="song"):
    """
    Add Spotify ids and metadata to chart songs.
    Songs that have charted are often on the chart the following week.
    As ids and metadata are added to multiple rows at a time.
    Make sure to skip rows that already have id and metadata.
    """
    for index, record in df.iterrows():
        artist, track = record[artist_key], record[track_key]
        # logging.info(f"artist: {artist}, track: {track}")
        filter_by_track_ = filter_by_track(artist_key=artist_key,
                                           track_key=track_key)

        # check if track id is already present
        if "spotify_track_id" in df\
                and len(filter_by_track_(df, artist, track)
                        & (df["spotify_track_id"].isna())) == 0:
            # logging.info("skipped: metadata exists")
            continue

        # search for song on spotify
        spotify_track = fetch_track(artist, track)
        if spotify_track is None:
            # logging.info(f"skipped: no track found: {artist} - {track}")
            continue

        # add id and metadata to dataframe
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_id"] = spotify_track["id"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_artist_id"] = spotify_track["artists"][0]["id"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_album_id"] = spotify_track["album"]["id"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_popularity"] = spotify_track["popularity"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_explicit"] = spotify_track["explicit"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_duration_ms"] = spotify_track["duration_ms"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_album_release_date"] =\
            spotify_track["album"]["release_date"]
        df.loc[filter_by_track_(df, artist, track),
               "spotify_track_album_release_date_precision"] =\
            spotify_track["album"]["release_date_precision"]

        # don't trigger spotify rate-limit
        sleep(1)
        if index % 10 == 0:
            total_tracks = len(df)
            null_tracks = df["spotify_track_id"].isnull().sum()
            print(f"status: {100 - null_tracks / total_tracks * 100:.2f} "
                  f"({total_tracks - null_tracks} / {total_tracks}")
    return df

# fetch_track("Rob Thomas & Santana", "Smooth")
# fetch_track("Rob Thomas Santana", "Smooth")
# fetch_track("Eiffel 65", "Blue (Da Ba Dee)")
# fetch_track("Missy Elliott", "Hot Boyz")
# fetch_track("P Diddy & R Kelly", "Satisfy You")
# fetch_track("Will Smith", "Will 2K")
# fetch_track("Ja Rule, Lil' Mo & Vita", "Put It On Me")
# fetch_track("Joe Thomas Mystikal", "Stutter")
# fetch_track("Ricky Martin & Christina Aguilera", "Nobody Wants To Be Lonely")
# fetch_track("ATC", "Around The World (La La La La La)")
song_artist_df_2 = with_spotify_track_metadata(song_artist_df_2)

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


status: 0.32 (4 / 1268
status: 0.39 (5 / 1268
status: 0.55 (7 / 1268
status: 1.26 (16 / 1268
status: 5.05 (64 / 1268
status: 6.07 (77 / 1268
status: 7.33 (93 / 1268
status: 7.65 (97 / 1268
status: 8.28 (105 / 1268
status: 9.23 (117 / 1268
status: 11.75 (149 / 1268
status: 12.38 (157 / 1268
status: 13.17 (167 / 1268
status: 14.43 (183 / 1268
status: 14.51 (184 / 1268
status: 14.91 (189 / 1268
status: 16.40 (208 / 1268
status: 17.67 (224 / 1268
status: 19.64 (249 / 1268
status: 20.90 (265 / 1268
status: 23.11 (293 / 1268
status: 23.34 (296 / 1268
status: 23.97 (304 / 1268
status: 24.84 (315 / 1268
status: 25.55 (324 / 1268
status: 26.10 (331 / 1268
status: 27.44 (348 / 1268
status: 28.86 (366 / 1268
status: 29.97 (380 / 1268
status: 30.91 (392 / 1268
status: 31.62 (401 / 1268
status: 32.33 (410 / 1268
status: 32.73 (415 / 1268
status: 35.09 (445 / 1268
status: 35.33 (448 / 1268
status: 36.67 (465 / 1268
status: 39.12 (496 / 1268
status: 39.35 (499 / 1268
status: 40.77 (517 / 1268
status:

In [48]:
song_artist_df_3 = song_artist_df_2[song_artist_df_2["spotify_track_id"].isnull()]
song_artist_df_3

,song,artist,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
74,The Great Beyond,REM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,I Wanna Know,Joe Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,There You Go,Pink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,Take A Look Around (Theme From MI 2),Limp Bizkit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,Faded,SoulDecision & Thrust,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12149,Slide,H.E.R. & YG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12191,Nobody But You,Blake Shelton Duet & Gwen Stefani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12376,I Hope,Gabby Barrett & Charlie Puth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12547,Hawai,Maluma & The Weeknd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
import re


def best_match(result, artist, track):
    artist = artist.replace(".", "")
    track = track.replace(".", "")
    for t in result["tracks"]["items"]:
        t_artist = t["artists"][0]["name"].replace(".", "")
        t_track = t["name"].replace(".", "")
        if "karaoke" in t_artist.lower() or "karaoke" in t_track.lower():
            continue
        if "version" in t_track.lower() and not "version" in track.lower():
            continue
        artist_fuzz_score = fuzz.token_set_ratio(artist, t_artist)
        track_fuzz_score = fuzz.token_set_ratio(track, t_track)
        compound_fuzz_score = fuzz.token_set_ratio(f"{artist} {track}", f"{t_artist} {t_track}")
        compound_ratio_fuzz_score = fuzz.ratio(f"{artist} {track}", f"{t_artist} {t_track}")
        # print(artist, t_artist, artist_fuzz_score, track, t_track, track_fuzz_score, compound_fuzz_score, compound_ratio_fuzz_score)
        if (artist_fuzz_score > 85 and track_fuzz_score > 85) or compound_fuzz_score > 90 or compound_ratio_fuzz_score > 90:
            return t
    return None


def fetch_track(artist, track):
    artists = [artist]
    if "," in artist:
        artists += artist.split(",")
    if artist.startswith("The "):
        artists.append(artist[4:])
    for artist in artists:
        if " & " in artist:
            artists += artist.split("&")
        try:
            # print(artist, track)
            artist_ = artist
            track_ =  re.sub("[\(\[].*?[\)\]]", "", track)
            result = spotify.search(f"track:{track} artist:{artist}")
            sleep(.25)
            best_match_ = best_match(result, artist, track)
            if result["tracks"]["total"] == 0 or not best_match_:
                artist_, _ = resolve_names(artist, track)
                result = spotify.search(f"track:{track} artist:{artist_}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist_} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist} {track_}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                artist_ = re.sub("'.*?'", "", artist)
                result = spotify.search(f"{artist_} {track}")
                best_match_ = best_match(result, artist_, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                continue
        except ReadTimeout:
            print("spotify time out.")
            continue
        if best_match_:
            return best_match_
    return None


# fetch_track("Limp Bizkit", "Take A Look Around (Theme From MI 2)")
# fetch_track("Ludacris & Shauna", "What's Your Fantasy")
# fetch_track("ATC", "Around The World (La La La La La)")
# fetch_track("Joe Thomas & Mystikal", "Stutter")
# fetch_track("QB Finest, Nas & Bravehearts", "Oochie Wally")
# fetch_track("Missy 'Misdemeanor' Elliott", "Get Ur Freak On")
# fetch_track("Christina Aguilera, Lil' Kim, Mya & Pink", "Lady Marmalade (Voulez-Vous Coucher Aver Moi Ce Soir?)")

song_artist_df_3 = with_spotify_track_metadata(song_artist_df_3)
song_artist_df_3

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


status: 15.23 (23 / 151
status: 23.18 (35 / 151
status: 33.77 (51 / 151
status: 35.10 (53 / 151
status: 48.34 (73 / 151
status: 50.33 (76 / 151


,song,artist,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
74,The Great Beyond,REM,7w57O4o0xCTn9YpKuaPZDd,4KWTAlx2RvbpseOGMEmROg,3UvVHQ31PhSMtI2GIS91Lz,54.0,False,307000.0,2003-01-01,day
84,I Wanna Know,Joe Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,There You Go,Pink,6dJ2mSRaKE9ctYw9qWNGWQ,1KCSPY1glIKqW2TotWuXOR,6PSOOxrBZuoLe88zn0wGQr,59.0,False,202800.0,2000-04-04,day
392,Take A Look Around (Theme From MI 2),Limp Bizkit,2avKuMN2QXkaG9vvHa2JLt,165ZgPlLkK7bf5bDoFc6Sb,5mi7FKaWE5CtcOjdyxScA7,76.0,False,321040.0,2000-10-17,day
399,Faded,SoulDecision & Thrust,4WP1XJnAUkZfTtMdZTaN9k,6hYrBYtoVGArYJcqvYxN4r,125budnbeWLmbMmMR6jV4t,53.0,False,208666.0,2000,year
...,...,...,...,...,...,...,...,...,...,...
12149,Slide,H.E.R. & YG,6MO2bfLHKykUgCChFdw91H,3Y7RZ31TRPVadSFVy1o8os,7LMJKPGcBZjZf6AEnTBS2t,76.0,False,296011.0,2020-01-17,day
12191,Nobody But You,Blake Shelton Duet & Gwen Stefani,5M8goiFYynmNxhueYW6grR,1UTPBmNbXNTittyMJrNkvw,4Kd33JPxTOcjTM29FrKoyT,70.0,False,194866.0,2019-12-13,day
12376,I Hope,Gabby Barrett & Charlie Puth,23T0OX7QOiIUFShSzbJ5Uo,6Iz3eq2aQGFf7TbGT2iahL,4Iqfx63CZhFGGIHiAvLxXY,79.0,False,210772.0,2020-06-19,day
12547,Hawai,Maluma & The Weeknd,1yoMvmasuxZfqHEipJhRbp,1r4hJ1h58CWwUQe3MxPuau,0p2yf6DucEgvj8Uk8KXJJv,81.0,False,199112.0,2020-08-21,day


In [82]:
song_artist_df_4 = song_artist_df_3[song_artist_df_3["spotify_track_id"].isnull()]
song_artist_df_4

,song,artist,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
84,I Wanna Know,Joe Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,He Loves U Not,Dream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,So In Love With Two,Micaela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,You Make Me Sick,Pink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,Bizounce,Olivia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
10117,Juju On Dat Beat (TZ Anthem),Zay Hilfiger & Zayion McCall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10286,How Far I'll Go,Alessia Cara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11342,I'll Never Love Again,Lady Gaga & Bradley Cooper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11739,Ran$om,Lil Tecca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
def best_match(result, artist, track):
    artist = artist.replace(".", "")
    track = track.replace(".", "")
    for t in result["tracks"]["items"]:
        t_artist = t["artists"][0]["name"].replace(".", "")
        t_track = t["name"].replace(".", "")
        if "karaoke" in t_artist.lower() or "karaoke" in t_track.lower():
            continue
        if "version" in t_track.lower() and not "version" in track.lower():
            continue
        artist_fuzz_score = fuzz.token_set_ratio(artist, t_artist)
        track_fuzz_score = fuzz.token_set_ratio(track, t_track)
        compound_fuzz_score = fuzz.token_set_ratio(f"{artist} {track}", f"{t_artist} {t_track}")
        compound_ratio_fuzz_score = fuzz.ratio(f"{artist} {track}", f"{t_artist} {t_track}")
        print(artist, t_artist, artist_fuzz_score, track, t_track, track_fuzz_score, compound_fuzz_score, compound_ratio_fuzz_score)
        if (artist_fuzz_score > 85 and track_fuzz_score > 85) or compound_fuzz_score > 90 or compound_ratio_fuzz_score > 90:
            return t
    return None


def fetch_track(artist, track):
    artists = [artist]
    if "," in artist:
        artists += artist.split(",")
    if artist.startswith("The "):
        artists.append(artist[4:])
    for artist in artists:
        if " & " in artist:
            artists += artist.split("&")
        try:
            # print(artist, track)
            artist_ = artist
            track_ =  re.sub("[\(\[].*?[\)\]]", "", track)
            result = spotify.search(f"track:{track} artist:{artist}")
            sleep(.25)
            best_match_ = best_match(result, artist, track)
            if result["tracks"]["total"] == 0 or not best_match_:
                artist_, _ = resolve_names(artist, track)
                result = spotify.search(f"track:{track} artist:{artist_}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist_} {track}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                result = spotify.search(f"{artist} {track_}")
                best_match_ = best_match(result, artist, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                artist_ = re.sub("'.*?'", "", artist)
                result = spotify.search(f"{artist_} {track}")
                best_match_ = best_match(result, artist_, track)
                sleep(.25)
            if result["tracks"]["total"] == 0 or not best_match_:
                continue
        except ReadTimeout:
            print("spotify time out.")
            continue
        if best_match_:
            return best_match_
    return None


fetch_track("T-Pain & Mike Jones", "I'm In Love with A Stripper")

T-Pain & Mike Jones Hei$enberg 30 I'm In Love with A Stripper Roller Coaster 29 23 11
T-Pain & Mike Jones Drew DeLeon 21 I'm In Love with A Stripper Tina 13 16 10
T-Pain & Mike Jones Jyourz 17 I'm In Love with A Stripper Move 19 14 17
T-Pain & Mike Jones Queso Tone 22 I'm In Love with A Stripper Dance Like You Work at the Club 41 37 31
T-Pain & Mike Jones Queso Tone 22 I'm In Love with A Stripper Dance Like You Work at the Club 41 37 31
T-Pain & Mike Jones Queso Tone 22 I'm In Love with A Stripper Dance Like You Work at the Club 41 37 31
T-Pain & Mike Jones Queso Tone 22 I'm In Love with A Stripper Dance Like You Work at the Club 41 37 31
T-Pain & Mike Jones Queso Tone 22 I'm In Love with A Stripper Dance Like You Work at the Club 41 37 31
T-Pain & Mike Jones Hei$enberg 30 I'm In Love with A Stripper Roller Coaster - Radio Edit 31 22 14
T-Pain  Hei$enberg 12 I'm In Love with A Stripper Roller Coaster 29 27 13
T-Pain  Drew DeLeon 24 I'm In Love with A Stripper Tina 13 20 12
T-Pain  Jyou

In [100]:
song_artist_df_all = pd.concat([
    song_artist_df,
    song_artist_df_2,
    song_artist_df_3,
])
song_artist_df_all = song_artist_df_all[~song_artist_df_all["spotify_track_id"].isnull()].drop_duplicates("spotify_track_id")
song_artist_df_all

,song,artist,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
1,What A Girl Wants,Christina Aguilera,5bGmuxShUba9maPswDnhCs,1l7ZsJRRS8wlW3WfJfPfNS,6fpPZS13ImRVpr7Tqs6yP9,67.0,False,215800.0,1999,year
2,I Knew I Loved You,Savage Garden,6nozDLxeL0TE4MS9GqYU1v,3NRFinRTEqUCfaTTZmk8ek,3w69KK7uEA8OsKuW3OeQcC,73.0,False,250360.0,1999-11-09,day
3,That's The Way It Is,Celine Dion,5s4catxeZsaWFnOrvrXZHf,4S9EykWXhStSc15wEx8QFK,51DfHvunyUysUZoBJEwbF8,69.0,False,241373.0,1999,year
5,Back At One,Brian McKnight,6mwA6YiKDjAUG8kWvRRUPh,6k0IBR0lU42s2GYpNX7kA9,0ZnKlkQrZb1xDbT8bmPSjo,74.0,False,263666.0,1999-09-21,day
6,Waiting For Tonight,Jennifer Lopez,5yaCquc7koPqtgj7v0lwHX,2DlGxzQSjYe5N6G9nkYghR,3Gby5NNeNYkMgAnrtEA3lc,65.0,False,246426.0,1999-06-01,day
...,...,...,...,...,...,...,...,...,...,...
11178,Jackie Chan,"Tiesto, Dzeko, Preme & Post Malone",4kWO6O1BUXcZmaxitpVUwp,2o5jDhtHVPhrJdv3cEQ99Z,0vRcQquqBlFvnezWldWfmt,76.0,True,215759.0,2018-05-18,day
11542,Walk Me Home,Pink,7DHApOHsIFo7EyfieArYGE,1KCSPY1glIKqW2TotWuXOR,2KKdF2S0u36Ug2jm2qTXlZ,71.0,False,177390.0,2019-02-20,day
12149,Slide,H.E.R. & YG,6MO2bfLHKykUgCChFdw91H,3Y7RZ31TRPVadSFVy1o8os,7LMJKPGcBZjZf6AEnTBS2t,76.0,False,296011.0,2020-01-17,day
12191,Nobody But You,Blake Shelton Duet & Gwen Stefani,5M8goiFYynmNxhueYW6grR,1UTPBmNbXNTittyMJrNkvw,4Kd33JPxTOcjTM29FrKoyT,70.0,False,194866.0,2019-12-13,day


In [101]:
pd.merge(songs_chart_by_month, song_artist_df_all, left_on=["song", "artist"], right_on=["song", "artist"], how="left")

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,...,spotify_track_album_release_date_x,spotify_track_album_release_date_precision_x,spotify_track_id_y,spotify_artist_id_y,spotify_album_id_y,spotify_track_popularity_y,spotify_track_explicit_y,spotify_track_duration_ms_y,spotify_track_album_release_date_y,spotify_track_album_release_date_precision_y
0,Jan 2000,1,Rob Thomas & Santana,Smooth,3911.953,1,-,44,-,-,...,2005-03-01,day,4qUfTdPwpeTKwck2ptbj4h,6GI52t8N5F02MxU0g5U69P,4gZuhUyDg1VRwosegw9pU2,26.0,False,233400.0,2003-12-15,day
1,Jan 2000,2,Christina Aguilera,What A Girl Wants,3787.015,1,81,18,-,1,...,1999,year,5bGmuxShUba9maPswDnhCs,1l7ZsJRRS8wlW3WfJfPfNS,6fpPZS13ImRVpr7Tqs6yP9,67.0,False,215800.0,1999,year
2,Jan 2000,3,Savage Garden,I Knew I Loved You,3693.844,1,56,68,43,1,...,1999-11-09,day,6nozDLxeL0TE4MS9GqYU1v,3NRFinRTEqUCfaTTZmk8ek,3w69KK7uEA8OsKuW3OeQcC,73.0,False,250360.0,1999-11-09,day
3,Jan 2000,4,Celine Dion,That's The Way It Is,3341.488,8,20,9,10,-,...,1999,year,5s4catxeZsaWFnOrvrXZHf,4S9EykWXhStSc15wEx8QFK,51DfHvunyUysUZoBJEwbF8,69.0,False,241373.0,1999,year
4,Jan 2000,5,Eiffel 65,Blue (Da Ba Dee),3340.978,6,28,22,10,1,...,2011-01-05,day,2yAVzRiEQooPEJ9SYx11L3,64rxQRJsLgZwHHyWKB8fiF,54vbD17F1t5q3yHkj1cX37,79.0,False,283747.0,2011-01-05,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,Jun 2021,46,Ariana Grande,POV,849.983,33,-,-,-,-,...,2020-10-30,day,3UoULw70kMsiVXxW0L3A33,66CXWjxzNUsdJxJ2JdwvnR,3euz4vS7ezKGnNSwgyvKcd,82.0,False,201882.0,2020-10-30,day
12896,Jun 2021,47,Dierks Bentley,Gone,843.855,32,-,-,-,85,...,2020-10-22,day,7IhfjcaXuXCq20ZiAA7igK,7x8nK0m0cP2ksQf0mjWdPS,7zEA1JpBgLz8Zj6cGcIvIV,69.0,False,205960.0,2020-10-22,day
12897,Jun 2021,48,Olivia Rodrigo,"Jealousy, Jealousy",838.863,24,-,-,-,21,...,2021-05-21,day,0MMyJUC3WNnFS1lit5pTjk,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,88.0,False,173160.0,2021-05-21,day
12898,Jun 2021,49,Olivia Rodrigo,"1 Step Forward, 3 Steps Back",837.277,19,-,-,-,17,...,2021-05-21,day,4wcBRRpIfesgcyUtis7PEg,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,82.0,True,163586.0,2021-05-21,day


In [104]:
songs_chart_by_month

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
0,Jan 2000,1,Rob Thomas & Santana,Smooth,3911.953,1,-,44,-,-,5,5IALWUYK0zDSEmZgb4ICvc,7BB7DAcx48BfzNf34VMr7y,6rkqnXiK12wQ8UthL6A52E,1.0,False,244924.0,2005-03-01,day
1,Jan 2000,2,Christina Aguilera,What A Girl Wants,3787.015,1,81,18,-,1,9,5bGmuxShUba9maPswDnhCs,1l7ZsJRRS8wlW3WfJfPfNS,6fpPZS13ImRVpr7Tqs6yP9,65.0,False,215800.0,1999,year
2,Jan 2000,3,Savage Garden,I Knew I Loved You,3693.844,1,56,68,43,1,21,6nozDLxeL0TE4MS9GqYU1v,3NRFinRTEqUCfaTTZmk8ek,3w69KK7uEA8OsKuW3OeQcC,71.0,False,250360.0,1999-11-09,day
3,Jan 2000,4,Celine Dion,That's The Way It Is,3341.488,8,20,9,10,-,14,5s4catxeZsaWFnOrvrXZHf,4S9EykWXhStSc15wEx8QFK,51DfHvunyUysUZoBJEwbF8,67.0,False,241373.0,1999,year
4,Jan 2000,5,Eiffel 65,Blue (Da Ba Dee),3340.978,6,28,22,10,1,1,2yAVzRiEQooPEJ9SYx11L3,64rxQRJsLgZwHHyWKB8fiF,54vbD17F1t5q3yHkj1cX37,77.0,False,283747.0,2011-01-05,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,Jun 2021,46,Ariana Grande,POV,849.983,33,-,-,-,-,-,3UoULw70kMsiVXxW0L3A33,66CXWjxzNUsdJxJ2JdwvnR,3euz4vS7ezKGnNSwgyvKcd,81.0,False,201882.0,2020-10-30,day
12896,Jun 2021,47,Dierks Bentley,Gone,843.855,32,-,-,-,85,-,7IhfjcaXuXCq20ZiAA7igK,7x8nK0m0cP2ksQf0mjWdPS,7zEA1JpBgLz8Zj6cGcIvIV,68.0,False,205960.0,2020-10-22,day
12897,Jun 2021,48,Olivia Rodrigo,"Jealousy, Jealousy",838.863,24,-,-,-,21,22,0MMyJUC3WNnFS1lit5pTjk,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,86.0,False,173160.0,2021-05-21,day
12898,Jun 2021,49,Olivia Rodrigo,"1 Step Forward, 3 Steps Back",837.277,19,-,-,-,17,18,4wcBRRpIfesgcyUtis7PEg,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,81.0,True,163586.0,2021-05-21,day


In [97]:
songs_chart_by_month = pd.read_parquet(data_path / "raw/song_chart_by_month.pq")
songs_chart_by_month.to_parquet(data_path / "raw/song_chart_by_month_v0.pq", index=False)

In [105]:
songs_chart_by_month = fetch_song_chart_by_month()
song_chart_by_month_v2 = pd.merge(songs_chart_by_month, song_artist_df_all, left_on=["song", "artist"], right_on=["song", "artist"], how="left")
song_chart_by_month_v2.to_parquet(data_path / "raw/song_chart_by_month.pq", index=False)

In [106]:
song_chart_by_month_v2

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
0,Jan 2000,1,Rob Thomas & Santana,Smooth,3911.953,1,-,44,-,-,5,4qUfTdPwpeTKwck2ptbj4h,6GI52t8N5F02MxU0g5U69P,4gZuhUyDg1VRwosegw9pU2,26.0,False,233400.0,2003-12-15,day
1,Jan 2000,2,Christina Aguilera,What A Girl Wants,3787.015,1,81,18,-,1,9,5bGmuxShUba9maPswDnhCs,1l7ZsJRRS8wlW3WfJfPfNS,6fpPZS13ImRVpr7Tqs6yP9,67.0,False,215800.0,1999,year
2,Jan 2000,3,Savage Garden,I Knew I Loved You,3693.844,1,56,68,43,1,21,6nozDLxeL0TE4MS9GqYU1v,3NRFinRTEqUCfaTTZmk8ek,3w69KK7uEA8OsKuW3OeQcC,73.0,False,250360.0,1999-11-09,day
3,Jan 2000,4,Celine Dion,That's The Way It Is,3341.488,8,20,9,10,-,14,5s4catxeZsaWFnOrvrXZHf,4S9EykWXhStSc15wEx8QFK,51DfHvunyUysUZoBJEwbF8,69.0,False,241373.0,1999,year
4,Jan 2000,5,Eiffel 65,Blue (Da Ba Dee),3340.978,6,28,22,10,1,1,2yAVzRiEQooPEJ9SYx11L3,64rxQRJsLgZwHHyWKB8fiF,54vbD17F1t5q3yHkj1cX37,79.0,False,283747.0,2011-01-05,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,Jun 2021,46,Ariana Grande,POV,849.983,33,-,-,-,-,-,3UoULw70kMsiVXxW0L3A33,66CXWjxzNUsdJxJ2JdwvnR,3euz4vS7ezKGnNSwgyvKcd,82.0,False,201882.0,2020-10-30,day
12896,Jun 2021,47,Dierks Bentley,Gone,843.855,32,-,-,-,85,-,7IhfjcaXuXCq20ZiAA7igK,7x8nK0m0cP2ksQf0mjWdPS,7zEA1JpBgLz8Zj6cGcIvIV,69.0,False,205960.0,2020-10-22,day
12897,Jun 2021,48,Olivia Rodrigo,"Jealousy, Jealousy",838.863,24,-,-,-,21,22,0MMyJUC3WNnFS1lit5pTjk,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,88.0,False,173160.0,2021-05-21,day
12898,Jun 2021,49,Olivia Rodrigo,"1 Step Forward, 3 Steps Back",837.277,19,-,-,-,17,18,4wcBRRpIfesgcyUtis7PEg,1McMsnEElThX1knmY4oliG,6s84u2TUpR3wdUv4NgKA2j,82.0,True,163586.0,2021-05-21,day
